In [1]:
import os
from langchain_openai import ChatOpenAI
from langgraph.graph import Graph
from langgraph.prebuilt import create_react_agent
import gradio as gr
from datetime import datetime

# Import tools
from tools.city_recommender import CityRecommendationTool
from tools.hotel_search import HotelSearchTool
from tools.flight_search import FlightSearchTool
from tools.attractions_tool import CityAttractionsTool
from city_vector_db import CityVectorDB

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
def initialize_tools():

    vector_db = CityVectorDB()
    vector_db.load("cities_vector_db")

    tools = [
        FlightSearchTool(api_key=os.getenv('SERPAPI_API_KEY')),
        CityAttractionsTool(),
        HotelSearchTool(api_key=os.getenv('SERPAPI_API_KEY')),
        CityRecommendationTool(vector_db=vector_db)
    ]
    return tools

In [3]:
tools = initialize_tools()

In [4]:
final_result = []
total_tokens = 0
prompt_tokens = 0
completion_tokens = 0
total_cost = 0.0

In [5]:
system_prompt = f"""You are a helpful travel assistant named Fred.

Follow these steps in order:

1. CITY RECOMMENDATION:
   - Use the city_recommendation_tool and the user's given preference EXACTLY ONCE to suggest a city
   
2. FLIGHT SEARCH:
   - Use the user's given source as the departure_airport and the first recommended city as the arrival_airport
   - Ask for specific travel dates (departure and return, if return is not given, ask for it and get the duration of the trip)
   - Use flight_search_tool with IATA codes
   - Format: {{
       'departure_airport': '[IATA code]',
       'arrival_airport': '[IATA code]',
       'departure_date': 'YYYY-MM-DD',
       'return_date': 'YYYY-MM-DD'  # Optional for one-way flights
   }}

3. HOTEL SEARCH
   - Look for hotels in the recommended city for the dates of the flight (departure and return date) and pick the cheapest one

4. ATTRACTIONS:
   - Once the hotel and flight are selected, use the city_attractions_tool to get the top attractions in the recommended city. Make sure you match the correct formats for the tool

5. END:
   - Once the user is done with the conversation, thank them and end the conversation
   
Remember that today's date is {datetime.now().strftime('%Y-%m-%d')}. Only search for future dates.
Do not repeat city recommendations unless explicitly asked by the user.

For eg. If the user's name is John Lincoln, they like skiing and are flying from New York. First, recommend a city using the city_recommendation_tool. Ask the user where they are flying from and the dates of travel if they haven't given it. Use the flight_search_tool to get the flight details.
Then look for hotels in the destination for the dates of the flight (departure and return date) using the hotel_search_tool. Then look for attractions in the destination using the city_attractions_tool. Inform the user of their whole itinerary..
"""

In [6]:
def create_agent(system_prompt):
    model = ChatOpenAI(model="gpt-4-turbo")
    tools = initialize_tools()
    return create_react_agent(model, tools=tools, state_modifier=system_prompt, debug=True)

In [7]:
agent = create_agent(system_prompt)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
from langchain.callbacks import get_openai_callback

messages = [("user", "my name is sam controlman and i want to plan a holiday from 25th to 30th december. I like skiing and am flying from Chicago.")]

with get_openai_callback() as cb:
    response = agent.invoke({"messages": messages})
    final_result.append(response['messages'][-1].content)
    total_tokens += cb.total_tokens
    prompt_tokens += cb.prompt_tokens
    completion_tokens += cb.completion_tokens
    total_cost += cb.total_cost
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost:.4f}")

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user',
               'my name is sam controlman and i want to plan a holiday from '
               '25th to 30th december. I like skiing and am flying from '
               'Chicago.')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user',
  'my name is sam controlman and i want to plan a holiday from 25th to 30th '
  'december. I like skiing and am flying from Chicago.')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='my name is sam controlman and i want to plan a holiday from 25th to 30th december. I like skiing and am flying from Chicago.', additional_kwargs={}, response_metadata={}, id='7b96f580-5fdc-4947-8562-2704cbf57871')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='my name is sam controlman and i want to plan a holiday from 25th

In [9]:
print(response['messages'][-1].content)

Here are the top attractions in Aspen to enhance your holiday experience:

1. **Maroon Bells**
   - **Category:** Adventure
   - **Description:** Stunning mountain peaks with picturesque hiking and photography opportunities.

2. **Aspen Mountain**
   - **Category:** Sport
   - **Description:** Popular for skiing and snowboarding with breathtaking views.

3. **Independence Pass**
   - **Category:** Adventure
   - **Description:** A scenic mountain pass with dramatic vistas and hiking trails.

4. **John Denver Sanctuary**
   - **Category:** Experiences
   - **Description:** A peaceful park with poetry-engraved rocks and serene landscapes.

5. **Aspen Art Museum**
   - **Category:** Experiences
   - **Description:** Contemporary art exhibits in a sleek, modern space.

6. **Snowmass Village**
   - **Category:** Sport
   - **Description:** Known for family-friendly skiing and year-round outdoor activities.

7. **Aspen Music Festival**
   - **Category:** Experiences
   - **Description:** Ren

In [10]:
message2 = [("user", "my name is rahul and i want to plan a holiday from 29th January 2025 to 15th february 2025. I like swimming and am flying from dubai.")]
message3 = [("user", "my name is ali. i like luxury. i want to plan a holiday from 19th february 2025 to 27th march 2025. I like luxury and am flying from riyadh.")]
message4 = [("user", "my name is soham. i like temples and old architecture. i want to plan a holiday from 1st march 2025 to 10th march 2025. I am flying from barcelona.")]
message5 = [("user", "my name is jacob and i want to plan a holiday from 29th January 2025 to 15th february 2025. I like old cities and am flying from buenos aires.")]

messages = [message2, message3, message4, message5]

In [11]:
for message in messages:
    with get_openai_callback() as cb:
        response = agent.invoke({"messages": message})
        final_result.append(response['messages'][-1].content)
        total_tokens += cb.total_tokens
        prompt_tokens += cb.prompt_tokens
        completion_tokens += cb.completion_tokens
        total_cost += cb.total_cost
        print(f"Total Tokens: {cb.total_tokens}")
        print(f"Prompt Tokens: {cb.prompt_tokens}")
        print(f"Completion Tokens: {cb.completion_tokens}")
        print(f"Total Cost (USD): ${cb.total_cost:.4f}")

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user',
               'my name is rahul and i want to plan a holiday from 29th '
               'January 2025 to 15th february 2025. I like swimming and am '
               'flying from dubai.')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user',
  'my name is rahul and i want to plan a holiday from 29th January 2025 to '
  '15th february 2025. I like swimming and am flying from dubai.')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='my name is rahul and i want to plan a holiday from 29th January 2025 to 15th february 2025. I like swimming and am flying from dubai.', additional_kwargs={}, response_metadata={}, id='4ec9fa90-be0e-421e-a105-704861a82fe3')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='my name is rahul and i want to plan a

In [12]:
final_result

['Here are the top attractions in Aspen to enhance your holiday experience:\n\n1. **Maroon Bells**\n   - **Category:** Adventure\n   - **Description:** Stunning mountain peaks with picturesque hiking and photography opportunities.\n\n2. **Aspen Mountain**\n   - **Category:** Sport\n   - **Description:** Popular for skiing and snowboarding with breathtaking views.\n\n3. **Independence Pass**\n   - **Category:** Adventure\n   - **Description:** A scenic mountain pass with dramatic vistas and hiking trails.\n\n4. **John Denver Sanctuary**\n   - **Category:** Experiences\n   - **Description:** A peaceful park with poetry-engraved rocks and serene landscapes.\n\n5. **Aspen Art Museum**\n   - **Category:** Experiences\n   - **Description:** Contemporary art exhibits in a sleek, modern space.\n\n6. **Snowmass Village**\n   - **Category:** Sport\n   - **Description:** Known for family-friendly skiing and year-round outdoor activities.\n\n7. **Aspen Music Festival**\n   - **Category:** Experien

In [13]:
print(total_tokens)
print(prompt_tokens)
print(completion_tokens)
print(total_cost)

30138
27271
2867
0.35872
